# 2. Baseline Model

Bu notebook en basit pipeline ve feature set ile temel bir model egitmeyi icerir.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report
import lightgbm as lgb

RANDOM_STATE = 42

## 1. Veri Hazirlama

In [ ]:
# df = pd.read_csv('../data/raw/train.csv')
# TARGET = 'target'
# X = df.drop(columns=[TARGET])
# y = df[TARGET]

## 2. Basit Preprocessing

In [ ]:
# Eksik degerleri doldur ve label encoding uygula

## 3. Baseline Model

In [ ]:
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
# model = lgb.LGBMClassifier(random_state=RANDOM_STATE, verbose=-1)
# scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc')
# print(f'Baseline AUC: {scores.mean():.4f}')

## 4. Sonuclar

- Baseline AUC: X.XXXX
- Sonraki adim: Feature Engineering